# Entrenamiento Por columna por FB Prophet
### Pasos:
1-. Importacion de librerias
2-. Division de Dataset (Marcado de huecos sinteticos)
3-. Calibracion de Parametros
4-. Entrenamiento de ML
5-. Generacion de Graficas de datos
6-. Pruebas de Precision

In [3]:
%pip install prophet
%pip install scikit-learn
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import itertools
import os 
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric



## FB Prophet Cross-Validation

In [2]:
import logging
logging.getLogger('fbprophet').setLevel(logging.ERROR)

from itertools import product
from prophet import Prophet

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

from tqdm import tqdm



c:\Users\Sarah\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### Hyperparameter Tuning para SECO

In [9]:
file_route = "../data_conagua_clasificada/Seco/NR_25037.csv"
output_route = "./hyperparameter_tuning_results/seco/"
os.makedirs(output_route, exist_ok=True)
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01,0.05, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0,5.0, 10.0]
}

columnas = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']

cutoffs = pd.to_datetime(['1965-06-01','1980-02-15', '2000-08-15', '2014-02-15'])

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
df = pd.read_csv(file_route)

for columna in columnas:
    rmsees = []
    if columna in df.columns:
        df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
        df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
        df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
        df_prop = df_prop.dropna(subset=['ds','y']).sort_values('ds').reset_index(drop=True)
        



    for params in tqdm(all_params):
        m = Prophet(**params)
        try:

            m = Prophet(**params).fit(df_prop)
            
            
            df_cv = cross_validation(m, cutoffs=cutoffs, horizon='365 days', parallel="processes")
            df_p = performance_metrics(df_cv,rolling_window=1)
            rmsee = df_p['rmse'].values[0]
            rmsees.append(rmsee)
        except Exception as e:
            rmsees.append(np.inf)
            print(f"Fallo con parámetros {params}: {e}")
            
            
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmsees

    print(tuning_results.sort_values('rmse').head())

    n_rows = 40
    display_df = tuning_results.sort_values('rmse').head(n_rows).copy()

    for c in display_df.select_dtypes(include=[float]).columns:
        display_df[c] = display_df[c].map(lambda x: f"{x:.4f}" if pd.notnull(x) and np.isfinite(x) else "")

    rows, cols = display_df.shape
    fig_w = max(8, min(24, 1.2 * cols))
    fig_h = max(2, 0.3 * rows)
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    ax.axis('off')

    table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.2)


    png_file = os.path.join(output_route, f"25037_{columna}_hyperparams.png")
    fig.tight_layout()
    fig.savefig(png_file, dpi=150, bbox_inches='tight')
    plt.close(fig)
    print(f"  Tabla de hiperparámetros guardada en: {png_file}")



  0%|          | 0/25 [00:00<?, ?it/s]20:46:38 - cmdstanpy - INFO - Chain [1] start processing
20:46:39 - cmdstanpy - INFO - Chain [1] done processing
  4%|▍         | 1/25 [00:07<03:09,  7.89s/it]20:46:46 - cmdstanpy - INFO - Chain [1] start processing
20:46:47 - cmdstanpy - INFO - Chain [1] done processing
  8%|▊         | 2/25 [00:16<03:10,  8.30s/it]20:46:54 - cmdstanpy - INFO - Chain [1] start processing
20:46:55 - cmdstanpy - INFO - Chain [1] done processing
 12%|█▏        | 3/25 [00:24<02:57,  8.08s/it]20:47:02 - cmdstanpy - INFO - Chain [1] start processing
20:47:03 - cmdstanpy - INFO - Chain [1] done processing
 16%|█▌        | 4/25 [00:32<02:54,  8.31s/it]20:47:11 - cmdstanpy - INFO - Chain [1] start processing
20:47:12 - cmdstanpy - INFO - Chain [1] done processing
 20%|██        | 5/25 [00:42<02:52,  8.65s/it]20:47:20 - cmdstanpy - INFO - Chain [1] start processing
20:47:22 - cmdstanpy - INFO - Chain [1] done processing
 24%|██▍       | 6/25 [00:51<02:49,  8.92s/it]20:47:30

    changepoint_prior_scale  seasonality_prior_scale      rmse
21                    0.500                      0.1  8.119761
22                    0.500                      1.0  8.120041
2                     0.001                      1.0  8.120932
24                    0.500                     10.0  8.120957
3                     0.001                      5.0  8.121225
  Tabla de hiperparámetros guardada en: ./hyperparameter_tuning_results/seco/25037_PRECIP_hyperparams.png


  0%|          | 0/25 [00:00<?, ?it/s]20:50:53 - cmdstanpy - INFO - Chain [1] start processing
20:50:56 - cmdstanpy - INFO - Chain [1] done processing
  4%|▍         | 1/25 [00:10<04:23, 10.99s/it]20:51:04 - cmdstanpy - INFO - Chain [1] start processing
20:51:08 - cmdstanpy - INFO - Chain [1] done processing
  8%|▊         | 2/25 [00:21<04:10, 10.89s/it]20:51:15 - cmdstanpy - INFO - Chain [1] start processing
20:51:18 - cmdstanpy - INFO - Chain [1] done processing
 12%|█▏        | 3/25 [00:32<03:57, 10.82s/it]20:51:26 - cmdstanpy - INFO - Chain [1] start processing
20:51:28 - cmdstanpy - INFO - Chain [1] done processing
 16%|█▌        | 4/25 [00:42<03:41, 10.56s/it]20:51:36 - cmdstanpy - INFO - Chain [1] start processing
20:51:38 - cmdstanpy - INFO - Chain [1] done processing
 20%|██        | 5/25 [00:52<03:28, 10.45s/it]20:51:46 - cmdstanpy - INFO - Chain [1] start processing
20:51:50 - cmdstanpy - INFO - Chain [1] done processing
 24%|██▍       | 6/25 [01:04<03:22, 10.68s/it]20:51:57

   changepoint_prior_scale  seasonality_prior_scale      rmse
0                    0.001                     0.01  2.315993
1                    0.001                     0.10  2.340757
4                    0.001                    10.00  2.341835
3                    0.001                     5.00  2.342101
2                    0.001                     1.00  2.343561
  Tabla de hiperparámetros guardada en: ./hyperparameter_tuning_results/seco/25037_EVAP_hyperparams.png


  0%|          | 0/25 [00:00<?, ?it/s]20:59:59 - cmdstanpy - INFO - Chain [1] start processing
21:00:01 - cmdstanpy - INFO - Chain [1] done processing
  4%|▍         | 1/25 [00:10<04:18, 10.78s/it]21:00:09 - cmdstanpy - INFO - Chain [1] start processing
21:00:12 - cmdstanpy - INFO - Chain [1] done processing
  8%|▊         | 2/25 [00:21<04:05, 10.67s/it]21:00:20 - cmdstanpy - INFO - Chain [1] start processing
21:00:23 - cmdstanpy - INFO - Chain [1] done processing
 12%|█▏        | 3/25 [00:32<03:58, 10.83s/it]21:00:31 - cmdstanpy - INFO - Chain [1] start processing
21:00:33 - cmdstanpy - INFO - Chain [1] done processing
 16%|█▌        | 4/25 [00:43<03:52, 11.05s/it]21:00:43 - cmdstanpy - INFO - Chain [1] start processing
21:00:46 - cmdstanpy - INFO - Chain [1] done processing
 20%|██        | 5/25 [00:56<03:56, 11.80s/it]21:00:56 - cmdstanpy - INFO - Chain [1] start processing
21:01:00 - cmdstanpy - INFO - Chain [1] done processing
 24%|██▍       | 6/25 [01:11<04:01, 12.72s/it]21:01:10

    changepoint_prior_scale  seasonality_prior_scale      rmse
20                    0.500                     0.01  3.367937
22                    0.500                     1.00  3.370890
21                    0.500                     0.10  3.371641
4                     0.001                    10.00  3.374743
23                    0.500                     5.00  3.375911
  Tabla de hiperparámetros guardada en: ./hyperparameter_tuning_results/seco/25037_TMAX_hyperparams.png


  0%|          | 0/25 [00:00<?, ?it/s]21:10:19 - cmdstanpy - INFO - Chain [1] start processing
21:10:22 - cmdstanpy - INFO - Chain [1] done processing
  4%|▍         | 1/25 [00:11<04:29, 11.23s/it]21:10:31 - cmdstanpy - INFO - Chain [1] start processing
21:10:33 - cmdstanpy - INFO - Chain [1] done processing
  8%|▊         | 2/25 [00:21<04:08, 10.82s/it]21:10:41 - cmdstanpy - INFO - Chain [1] start processing
21:10:44 - cmdstanpy - INFO - Chain [1] done processing
 12%|█▏        | 3/25 [00:34<04:18, 11.74s/it]21:10:54 - cmdstanpy - INFO - Chain [1] start processing
21:10:58 - cmdstanpy - INFO - Chain [1] done processing
 16%|█▌        | 4/25 [00:50<04:42, 13.46s/it]21:11:10 - cmdstanpy - INFO - Chain [1] start processing
21:11:16 - cmdstanpy - INFO - Chain [1] done processing
 20%|██        | 5/25 [01:10<05:17, 15.90s/it]21:11:31 - cmdstanpy - INFO - Chain [1] start processing
21:11:37 - cmdstanpy - INFO - Chain [1] done processing
 24%|██▍       | 6/25 [01:31<05:34, 17.62s/it]21:11:52

    changepoint_prior_scale  seasonality_prior_scale      rmse
20                      0.5                     0.01  3.034577
21                      0.5                     0.10  3.051406
22                      0.5                     1.00  3.052985
24                      0.5                    10.00  3.053302
23                      0.5                     5.00  3.054134
  Tabla de hiperparámetros guardada en: ./hyperparameter_tuning_results/seco/25037_TMIN_hyperparams.png


## Performance Metrics

### METRICS EVAP

In [9]:
file_route = "../data_conagua_clasificada/Seco/NR_25037.csv"
output_route = "./Performance_metrics_plots/seco/NR_25037/"
os.makedirs(output_route, exist_ok=True)


#Parametros para EVAP
param_grid = {  
    'changepoint_prior_scale': [0.001],
    'seasonality_prior_scale': [0.01]
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
df = pd.read_csv(file_route)

columna = 'EVAP'

df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
df_prop = df_prop.dropna(subset=['ds','y']).sort_values('ds').reset_index(drop=True)

for params in all_params:
    m = Prophet(**params)
    try:
        m = Prophet(**params).fit(df_prop)
        
        
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days', parallel="processes")
        
        display(df_cv.head())
        
        
        df_p = performance_metrics(df_cv)
        print(df_p.head())
        
        n_rows = 40
        display_df = df_p.head(n_rows).copy()
        for c in display_df.select_dtypes(include=[float]).columns:
            display_df[c] = display_df[c].map(lambda x: f"{x:.4f}" if pd.notnull(x) and np.isfinite(x) else "")
        rows, cols = display_df.shape
        fig_w = max(8, min(24, 1.2 * cols))
        fig_h = max(2, 0.3 * rows)
        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        ax.axis('off')
        table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 1.2)
        png_file = os.path.join(output_route, f"25037_{columna}_performance_metrics.png")
        fig.tight_layout()
        fig.savefig(png_file, dpi=150, bbox_inches='tight')
        plt.close(fig)
        print(f"  Tabla de métricas de desempeño guardada en: {png_file}")
        
        
    except Exception as e:
        print(f"Fallo con parámetros {params}: {e}")


00:00:20 - cmdstanpy - INFO - Chain [1] start processing
00:00:26 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1959-01-01,4.576430,2.148310,6.867581,2.17,1958-06-16
1,1959-01-02,4.389354,2.048062,6.779391,3.89,1958-06-16
2,1959-01-03,4.365116,2.021666,6.637494,4.12,1958-06-16
3,1959-01-04,4.297404,1.947922,6.731689,1.78,1958-06-16
4,1959-01-05,4.034178,1.802552,6.519102,0.61,1958-06-16


  horizon       mse      rmse       mae      mape     mdape     smape  \
0 37 days  3.887949  1.971788  1.469979  0.387861  0.178325  0.249272   
1 38 days  3.884563  1.970930  1.464925  0.385529  0.176878  0.247812   
2 39 days  3.863215  1.965506  1.458024  0.388261  0.175234  0.246988   
3 40 days  3.851121  1.962427  1.453908  0.389067  0.173019  0.246226   
4 41 days  3.865132  1.965994  1.455690  0.393202  0.172577  0.246861   

   coverage  
0  0.827000  
1  0.828132  
2  0.829894  
3  0.831203  
4  0.832289  
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/seco/NR_25037/25037_EVAP_performance_metrics.png


### METRICS PRECIP

In [11]:
file_route = "../data_conagua_clasificada/Seco/NR_25037.csv"
output_route = "./Performance_metrics_plots/seco/NR_25037/"
os.makedirs(output_route, exist_ok=True)


#Parametros para PRECIP
param_grid = {  
    'changepoint_prior_scale': [0.5],
    'seasonality_prior_scale': [0.1]
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
df = pd.read_csv(file_route)

columna = 'PRECIP'

df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
df_prop = df_prop.dropna(subset=['ds','y']).sort_values('ds').reset_index(drop=True)

for params in all_params:
    m = Prophet(**params)
    try:
        m = Prophet(**params).fit(df_prop)
        
        
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days', parallel="processes")
        
        display(df_cv.head())
        
        
        df_p = performance_metrics(df_cv)
        print(df_p.head())
        
        n_rows = 40
        display_df = df_p.head(n_rows).copy()
        for c in display_df.select_dtypes(include=[float]).columns:
            display_df[c] = display_df[c].map(lambda x: f"{x:.4f}" if pd.notnull(x) and np.isfinite(x) else "")
        rows, cols = display_df.shape
        fig_w = max(8, min(24, 1.2 * cols))
        fig_h = max(2, 0.3 * rows)
        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        ax.axis('off')
        table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 1.2)
        png_file = os.path.join(output_route, f"25037_{columna}_performance_metrics.png")
        fig.tight_layout()
        fig.savefig(png_file, dpi=150, bbox_inches='tight')
        plt.close(fig)
        print(f"  Tabla de métricas de desempeño guardada en: {png_file}")
        
        
    except Exception as e:
        print(f"Fallo con parámetros {params}: {e}")


00:05:22 - cmdstanpy - INFO - Chain [1] start processing
00:05:41 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1959-01-01,2.751463,-5.940510,10.962173,0.0,1958-01-19
1,1959-01-02,2.055270,-6.272735,10.423591,0.0,1958-01-19
2,1959-01-03,2.099441,-6.478212,10.439680,0.0,1958-01-19
3,1959-01-04,2.530893,-6.085705,10.606883,0.0,1958-01-19
4,1959-01-05,2.870535,-5.500320,10.487709,0.5,1958-01-19


  horizon        mse      rmse       mae  mdape     smape  coverage
0 37 days  53.601518  7.321306  2.675920    inf  1.871068  0.958626
1 38 days  52.497024  7.245483  2.658990    inf  1.871099  0.958647
2 39 days  52.457342  7.242744  2.658133    inf  1.872273  0.958421
3 40 days  51.775913  7.195548  2.639641    inf  1.871578  0.958929
4 41 days  51.742000  7.193191  2.642877    inf  1.871946  0.958793
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/seco/NR_25037/25037_PRECIP_performance_metrics.png


## METRICS TMAX

In [13]:
file_route = "../data_conagua_clasificada/Seco/NR_25037.csv"
output_route = "./Performance_metrics_plots/seco/NR_25037/"
os.makedirs(output_route, exist_ok=True)


#Parametros para TMAX
param_grid = {  
    'changepoint_prior_scale': [0.5],
    'seasonality_prior_scale': [0.01]
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
df = pd.read_csv(file_route)

columna = 'TMAX'

df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
df_prop = df_prop.dropna(subset=['ds','y']).sort_values('ds').reset_index(drop=True)

for params in all_params:
    m = Prophet(**params)
    try:
        m = Prophet(**params).fit(df_prop)
        
        
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days', parallel="processes")
        
        display(df_cv.head())
        
        
        df_p = performance_metrics(df_cv)
        print(df_p.head())
        
        n_rows = 40
        display_df = df_p.head(n_rows).copy()
        for c in display_df.select_dtypes(include=[float]).columns:
            display_df[c] = display_df[c].map(lambda x: f"{x:.4f}" if pd.notnull(x) and np.isfinite(x) else "")
        rows, cols = display_df.shape
        fig_w = max(8, min(24, 1.2 * cols))
        fig_h = max(2, 0.3 * rows)
        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        ax.axis('off')
        table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 1.2)
        png_file = os.path.join(output_route, f"25037_{columna}_performance_metrics.png")
        fig.tight_layout()
        fig.savefig(png_file, dpi=150, bbox_inches='tight')
        plt.close(fig)
        print(f"  Tabla de métricas de desempeño guardada en: {png_file}")
        
        
    except Exception as e:
        print(f"Fallo con parámetros {params}: {e}")


00:13:15 - cmdstanpy - INFO - Chain [1] start processing
00:13:32 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1959-01-01,19.217768,15.684675,22.384415,25.0,1958-01-19
1,1959-01-02,18.845494,15.372631,22.204831,28.0,1958-01-19
2,1959-01-03,18.573134,15.455478,21.972672,27.5,1958-01-19
3,1959-01-04,18.522984,15.261713,21.657322,23.5,1958-01-19
4,1959-01-05,17.838853,14.648556,20.936157,19.0,1958-01-19


  horizon       mse      rmse       mae      mape     mdape     smape  \
0 37 days  8.233272  2.869368  2.175864  0.069625  0.051401  0.068214   
1 38 days  8.298040  2.880632  2.184737  0.069968  0.051900  0.068527   
2 39 days  8.355648  2.890614  2.190821  0.070180  0.052296  0.068727   
3 40 days  8.366131  2.892427  2.193166  0.070207  0.052296  0.068791   
4 41 days  8.370730  2.893221  2.190019  0.070181  0.051719  0.068741   

   coverage  
0  0.819489  
1  0.818511  
2  0.818157  
3  0.817633  
4  0.817146  
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/seco/NR_25037/25037_TMAX_performance_metrics.png


## METRICS TMIN

In [14]:
file_route = "../data_conagua_clasificada/Seco/NR_25037.csv"
output_route = "./Performance_metrics_plots/seco/NR_25037/"
os.makedirs(output_route, exist_ok=True)


#Parametros para TMIN
param_grid = {  
    'changepoint_prior_scale': [0.5],
    'seasonality_prior_scale': [0.01]
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
df = pd.read_csv(file_route)

columna = 'TMIN'

df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
df_prop = df_prop.dropna(subset=['ds','y']).sort_values('ds').reset_index(drop=True)

for params in all_params:
    m = Prophet(**params)
    try:
        m = Prophet(**params).fit(df_prop)
        
        
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days', parallel="processes")
        
        display(df_cv.head())
        
        
        df_p = performance_metrics(df_cv)
        print(df_p.head())
        
        n_rows = 40
        display_df = df_p.head(n_rows).copy()
        for c in display_df.select_dtypes(include=[float]).columns:
            display_df[c] = display_df[c].map(lambda x: f"{x:.4f}" if pd.notnull(x) and np.isfinite(x) else "")
        rows, cols = display_df.shape
        fig_w = max(8, min(24, 1.2 * cols))
        fig_h = max(2, 0.3 * rows)
        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        ax.axis('off')
        table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 1.2)
        png_file = os.path.join(output_route, f"25037_{columna}_performance_metrics.png")
        fig.tight_layout()
        fig.savefig(png_file, dpi=150, bbox_inches='tight')
        plt.close(fig)
        print(f"  Tabla de métricas de desempeño guardada en: {png_file}")
        
        
    except Exception as e:
        print(f"Fallo con parámetros {params}: {e}")


00:28:35 - cmdstanpy - INFO - Chain [1] start processing
00:28:52 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1959-01-01,-40.023826,-43.358097,-36.572833,3.5,1958-01-19
1,1959-01-02,-39.918927,-43.122548,-36.669247,6.0,1958-01-19
2,1959-01-03,-39.947381,-43.286446,-36.609761,7.0,1958-01-19
3,1959-01-04,-39.974052,-43.393209,-36.723908,8.5,1958-01-19
4,1959-01-05,-39.689726,-42.903436,-36.471388,7.5,1958-01-19


  horizon       mse      rmse       mae      mape     mdape     smape  \
0 37 days  8.876513  2.979348  2.099915  0.158185  0.087540  0.138341   
1 38 days  8.966384  2.994392  2.111393  0.159202  0.087896  0.139138   
2 39 days  9.020158  3.003358  2.118775  0.159989  0.088391  0.139789   
3 40 days  9.061283  3.010197  2.124941  0.160553  0.088840  0.140215   
4 41 days  9.079954  3.013296  2.128447  0.160661  0.088916  0.140370   

   coverage  
0  0.863844  
1  0.860921  
2  0.858865  
3  0.857851  
4  0.857313  
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/seco/NR_25037/25037_TMIN_performance_metrics.png
